In [1]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import sys
sys.path.insert(0, os.path.abspath(os.path.join("..", "src")))
from modeling import *

In [2]:
df = pd.read_csv("../data/train.csv")
y = df['target']
X = df['text']  

In [3]:
vocab_size = 10000  
maxlen = 200  

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=maxlen)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [6]:
BATCH_SIZE = 16
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [7]:
embed_dim = 16  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 16  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x) 



In [8]:
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(train_dataset,epochs=3, validation_data=test_dataset)


test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

Epoch 1/3


405/405 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.5318 - loss: 0.7005 - val_accuracy: 0.5701 - val_loss: 0.6799
Epoch 2/3
405/405 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5757 - loss: 0.6730 - val_accuracy: 0.7802 - val_loss: 0.5119
Epoch 3/3
405/405 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8126 - loss: 0.4390 - val_accuracy: 0.8109 - val_loss: 0.4622
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8173 - loss: 0.4639
Test Loss: 0.4622330665588379, Test Accuracy: 0.8108581304550171


In [9]:
from sklearn.metrics import classification_report, f1_score
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

# Generate classification report
print(classification_report(y_test, y_pred, digits=4))

f1 = f1_score(y_pred,y_test)
print(f"F1 Score: {f1:.4f}")
# Optionally, evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
              precision    recall  f1-score   support

           0     0.8085    0.8756    0.8407       651
           1     0.8146    0.7251    0.7672       491

    accuracy                         0.8109      1142
   macro avg     0.8116    0.8003    0.8040      1142
weighted avg     0.8111    0.8109    0.8091      1142

F1 Score: 0.7672
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8173 - loss: 0.4639
Test Loss: 0.4622330665588379, Test Accuracy: 0.8108581304550171
